# Visualizing Image Data

This notebook will demonstrate how to load and visualize astronomical images in the pywwt viewer.

---

## Step 1: Setting up WWT

First, we need to connect to the WWT app. If WWT isn't running yet, activate the JupyterLab command palette from the View menu, and select "AAS WorldWide Telescope". Then all we need to do is run:

In [ ]:
from pywwt.jupyter import connect_to_app
wwt = await connect_to_app().becomes_ready()

Now the `wwt` variable will let us talk to the WWT app. For example, the following command will make some things a bit easier for us farther down in this tutorial:

In [ ]:
wwt.foreground_opacity = 0

We'll also set up a utility function to help us load data files stored alongside this notebook:

In [ ]:
def datapath(*args):
    from os.path import join
    return join('data', *args)

---

## Step 2: Visualizing a small FITS image

We'll start by visualizing a WISE 12µm image towards the [Westerhout 5 star forming region](https://en.wikipedia.org/wiki/Westerhout_5) and taking a look at some of the advanced visualization options.

Images, like data tables, are represented in WWT as "layers" that can be added to the view. With a standard FITS file, all you need to do is provide a file name:

In [ ]:
layer = wwt.layers.add_image_layer(datapath('w5.fits'))

The viewer will automatically center and zoom to the image you've loaded.

Once you've loaded a FITS image this way, WWT provides a standard set of controls over how exactly it's displayed. Clicking on the panel item labeled "w5" will reveal tools for controlling aspects of the image display like its opacity and the colormap. Clicking the target icons next to the "Low cutoff" and "High cutoff" settings will enter an interactive mode for adjusting these settings: clicking towards the top-left or top-right of the WWT window will substantially decrease or increase these parameters, respectively.

For instance, by sliding the Opacity control, you can see how the image compares to the background survey. For this example, a good comparison is the IRAS dataset:

In [ ]:
wwt.background = 'IRIS: Improved Reprocessing of IRAS Survey (Infrared)'

You can also control the same settings from Python. This code changes a bunch of them at once:

In [ ]:
layer.cmap = 'plasma'
layer.vmin = 400
layer.vmax = 1000
layer.stretch = 'sqrt'
layer.opacity = 0.9

Note that there are currently some limitations in pywwt that limit the synchronization between the Python code and the WWT app, so mixing and matching the two interfaces can sometimes lead to unexpected results.

After you're done playing around, let's reset the WWT widget:

In [ ]:
wwt.reset()

---

# Step 3: Visualizing big images

The image was used above was pretty, but small. If your data are bigger, WWT still has you covered.

Here we'll visualize a larger image from the NRAO [VLASS][vlass] survey. The image we use has been specially processed to (1) be a bigger a challenge than the one before, but (2) still be usable on the very resource-constrained virtual machines that host our demonstration cloud notebooks. In actual work, when your kernel isn't running on a free temporary cloud service, resource limitations should almost never be an issue.

[vlass]: https://science.nrao.edu/vlass

First, we'll download the sample image to your kernel. This may take a little while.

In [ ]:
from astropy.utils.data import download_file

url = "https://data1.wwtassets.org/packages/2022/01_demos/vlass2.1-ql-T01t30-J145214-363000.fits"
path = download_file(url)

We can check and see that this file is more than 100 megabytes large:

In [ ]:
!ls -lh $path

If you had to wait to download the whole thing every time you wanted to view it, that would be pretty annoying!

To view this file in WWT, we use the same pattern as above. This time, however, pywwt does some extra processing ("tiling") to enable quick visualization:

In [ ]:
layer = wwt.layers.add_image_layer(path)

This image contains [a pretty double radio galaxy][x] along the lower left edge which you can pull out by adjusting the data cuts. These settings are a good starting point:

[x]: https://public.nrao.edu/gallery/radio-galaxy/

In [ ]:
layer.vmin = -0.002
layer.vmax = 0.009

---

## Step 4: Layering images

One of the strengths of the WWT viewer is that you can layer as many images as you'd like and compare them. This can be great for multiwavelength science. As an example, let's use the Python module [astroquery](https://astroquery.readthedocs.io/en/latest/) to fetch 2MASS Ks-band images of the field of supernova 2011fe. This might take a little while since, like the VLASS example before, the Python kernel needs to download the data from MAST.

In [ ]:
from astroquery.skyview import SkyView

img_list = SkyView.get_images(
    position='SN 2011FE', 
    survey='2MASS-K', 
    pixels=700  # you can adjust the size if you want
)
assert len(img_list) == 1  # there's only one matching item in this example
twomass_img = img_list[0]
twomass_img.info()


Once the FITS data are available, we can display them in pywwt using the same pattern as before. (The `add_image_layer()` function accepts either filenames or Astropy FITS data objects.)

In [ ]:
twomass_layer = wwt.layers.add_image_layer(twomass_img)

Once again you should see the view automatically center on your image.

Now we'll load up another image of the same field that came from *Swift*, this time stored as a local file as in the first example:

In [ ]:
swift_layer = wwt.layers.add_image_layer(datapath('m101_swiftx.fits'))

Now you can use the app's controls to fade in and out the different wavelengths.

Try changing the background imagery to the PanSTARRS1 3pi survey, and see if you can find a point source that's bright in the optical, IR, *and* X-ray wavebands. Click on [its SIMBAD page][simbad] to find out what kind of object it is!

[simbad]: http://simbad.u-strasbg.fr/simbad/sim-id?Ident=%408686502&Name=CXO%20J140312.4%2b541753

---

## Next Steps

To learn how to display data tables along with your imagery, start with the [NASA Exoplanet Archive](./NASA%20Exoplanet%20Archive.ipynb) tutorial

---

## Credits

This notebook was prepared by:

- O. Justin Otor
- Thomas Robitaille
- Peter K. G. Williams